In [16]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [17]:
!huggingface-cli whoami

hieuhatinh


### Aspect Term Extraction

#### Build Dataset

In [1]:
!pip install -q datasets==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [2]:
from datasets import load_dataset
ds = load_dataset('thainq107/abte-restaurants')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/454 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/183k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3602 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1119 [00:00<?, ? examples/s]

#### Tokenization

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

def tokenize_and_align_labels(examples):
    tokenized_inputs = []
    labels = []
    for tokens, tags in zip(examples['Tokens'], examples['Tags']):
        # tokens = ' '.join(tokens).replace("'", "").strip('][').split(', ')
        # tags = ' '.join(tags).strip('][').split(', ')
        bert_tokens = []
        bert_tags = []

        for i in range(len(tokens)):
            t = tokenizer.tokenize(tokens[i])
            bert_tokens += t
            bert_tags += [int(tags[i])] * len(t)

        bert_ids = tokenizer.convert_tokens_to_ids(bert_tokens)

        tokenized_inputs.append(bert_ids)
        labels.append(bert_tags)
    return {
        'input_ids': tokenized_inputs,
        'labels': labels
    }

preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [4]:
preprocessed_ds

DatasetDict({
    train: Dataset({
        features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'labels'],
        num_rows: 3602
    })
    test: Dataset({
        features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'labels'],
        num_rows: 1119
    })
})

#### Data Collator

In [5]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

#### Evaluate

In [6]:
!pip install -q seqeval==1.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [7]:
import numpy as np
from seqeval.metrics import accuracy_score

def compute_metrics(p):
    predictions, labels = p
    prediction = np.argmax(predictions, axis=2)

    true_predictions = [
        [str(p) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [str(l) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label  in zip(predictions, labels)
    ]
    results = accuracy_score(true_predictions, true_labels)
    return {'accuracy': results}

#### Model

In [8]:
from transformers import AutoModelForTokenClassification

id2label = {
    0: 'O',
    1: 'B-Term',
    2: 'I-Term'
}
label2id = {
    'O': 0,
    'B-Term': 1,
    'I-Term': 2
}

model = AutoModelForTokenClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Training

In [9]:
import os
os.environ['WANDB_DISABLED'] = 'true'
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='abte-restaurants-distilbert-base-uncased',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds['train'],
    eval_dataset=preprocessed_ds['test'],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.175445,0.000000
2,No log,0.191602,0.000000
3,0.140600,0.207828,0.000000
4,0.140600,0.220817,0.000000
5,0.030700,0.228730,0.000000


TrainOutput(global_step=1130, training_loss=0.0785406608497147, metrics={'train_runtime': 158.5851, 'train_samples_per_second': 113.567, 'train_steps_per_second': 7.126, 'total_flos': 241258883931900.0, 'train_loss': 0.0785406608497147, 'epoch': 5.0})

In [23]:
model.push_to_hub("hieuhatinh/abte-restaurants-distilbert-base-uncased")
tokenizer.push_to_hub("hieuhatinh/abte-restaurants-distilbert-base-uncased")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/hieuhatinh/abte-restaurants-distilbert-base-uncased/commit/c220aa3eae6c731c09460e1c5967819ca9a859a6', commit_message='Upload tokenizer', commit_description='', oid='c220aa3eae6c731c09460e1c5967819ca9a859a6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hieuhatinh/abte-restaurants-distilbert-base-uncased', endpoint='https://huggingface.co', repo_type='model', repo_id='hieuhatinh/abte-restaurants-distilbert-base-uncased'), pr_revision=None, pr_num=None)

#### Prediction

In [24]:
from transformers import pipeline

token_classifier = pipeline(
    model='hieuhatinh/abte-restaurants-distilbert-base-uncased',
    aggregation_strategy='simple'
)

test_sentence = 'The bread is top notch as well'
results = token_classifier(test_sentence)

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


### Aspect Term Sentiment Classification

#### Build Dataset

In [25]:
!pip install -q datasets==3.2.0

In [26]:
from datasets import load_dataset
ds = load_dataset('thainq107/abte-restaurants')

#### Tokenization

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

def tokenize_and_align_labels(examples):
    sentences, sentence_tags = [], []
    labels = []
    for tokens, pols in zip(examples['Tokens'], examples['Polarities']):
        # tokens = tokens.replace("'", "").strip('][').split(', ')
        # pols = pols.strip('][').split(', ')

        bert_tokens = []
        bert_att = []
        pols_label = 0

        for i in range(len(tokens)):
            t = tokenizer.tokenize(tokens[i])
            bert_tokens += t
            if int(pols[i]) != -1:
                bert_att += t
                pols_label = int(pols[i])

        sentences.append(' '.join(bert_tokens))
        sentence_tags.append(' '.join(bert_att))
        labels.append(pols_label)

    tokenized_inputs = tokenizer(sentences, sentence_tags, padding=True, truncation=True, return_tensors='pt')
    tokenized_inputs['labels'] = labels
    return tokenized_inputs

preprocessed_ds = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1119 [00:00<?, ? examples/s]

In [28]:
preprocessed_ds

DatasetDict({
    train: Dataset({
        features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3602
    })
    test: Dataset({
        features: ['Tokens', 'Tags', 'Polarities', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1119
    })
})

#### Evaluate

In [29]:
!pip install -q evaluate==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00


In [30]:
import numpy as np
import evaluate

accuracy = evaluate.load('accuracy')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

#### Model

In [31]:
from transformers import AutoModelForSequenceClassification

id2label = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
laebel2id = {'Negative': 0, 'Neutral': 1, 'Positive': 2}

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased', num_labels=3, id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Training

In [32]:
import os
os.environ['WANDB_DISABLED'] = 'true'
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='absa-restaurants-distilbert-base-uncased',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.1,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds['train'],
    eval_dataset=preprocessed_ds['test'],
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.562326,0.787310
2,No log,0.520843,0.801609
3,0.584000,0.501082,0.819482
4,0.584000,0.523461,0.823056
5,0.289200,0.533453,0.820375


TrainOutput(global_step=1130, training_loss=0.40882997934797166, metrics={'train_runtime': 287.863, 'train_samples_per_second': 62.564, 'train_steps_per_second': 3.925, 'total_flos': 689416103009232.0, 'train_loss': 0.40882997934797166, 'epoch': 5.0})

In [35]:
model.push_to_hub("hieuhatinh/absa-restaurants-distilbert-base-uncased")
tokenizer.push_to_hub("hieuhatinh/absa-restaurants-distilbert-base-uncased")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/hieuhatinh/absa-restaurants-distilbert-base-uncased/commit/ac37d392347a5e9aa29a48a227750d78f8504010', commit_message='Upload tokenizer', commit_description='', oid='ac37d392347a5e9aa29a48a227750d78f8504010', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hieuhatinh/absa-restaurants-distilbert-base-uncased', endpoint='https://huggingface.co', repo_type='model', repo_id='hieuhatinh/absa-restaurants-distilbert-base-uncased'), pr_revision=None, pr_num=None)

#### Prediction

In [39]:
from transformers import pipeline

token_classifier = pipeline(
    model='hieuhatinh/abte-restaurants-distilbert-base-uncased',
    aggregation_strategy='simple'
)

classifier = pipeline(
    model='hieuhatinh/absa-restaurants-distilbert-base-uncased',
)

test_sentence = 'The bread is top notch as well but the milk tee is terrible'
results = token_classifier(test_sentence)
sentence_tags = ' '.join([result['word'] for result in results])
pred_label = classifier(f'{test_sentence} [SEP] {sentence_tags}')
sentence_tags, pred_label

Device set to use cuda:0
Device set to use cuda:0


('bread milk tee', [{'label': 'Negative', 'score': 0.9131714105606079}])